In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# traindf = pd.read_pickle("./pickles/df_small_train.pkl")
# testdf = pd.read_pickle("./pickles/df_small_test.pkl")
# nep_testdf = pd.read_pickle("./pickles/df_small_train_test_yk.pkl")

# run for a very small dataset for testing
# traindf = pd.read_pickle("./pickles/df_verysmall_train_test_yk.pkl")


# run if you want to use the whole train/test dataset
# traindf = pd.read_csv("data/test_set_VU_DM.csv")
traindf.head()

# filename = "clean_test_set_VU_DM"

In [3]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor and there are available rooms.
    1 is True, 0 is False.
    """
    competitor_bools = []
    availability_bools = []
    
    # Columns for existing competitors
    compare_cols = ["comp1_rate", "comp2_rate", "comp3_rate", "comp4_rate", "comp5_rate",
                "comp6_rate", "comp7_rate", "comp8_rate"]
    
    # Columns for room availability
    availability_cols = ["comp1_inv", "comp2_inv", "comp3_inv", "comp4_inv", "comp5_inv", 
                        "comp6_inv", "comp7_inv", "comp8_inv"]

    for index, row in df.iterrows():
        comp_bool = 0
        available = 0

        # Check for a combination of competitor price and room-availability. 
        for competitor, availability in zip(compare_cols, availability_cols):
            if row[competitor] == 1: # and row[availability] == 1: TODO!!!
                comp_bool = 1
            if row[availability] == 1:
                available = 1
        
        competitor_bools.append(comp_bool)
        availability_bools.append(available)
        
    # Append list of bools to new column
    df["competitor_bool"] = competitor_bools
    df["availability_bools"] = availability_bools
    
    return df

In [4]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # Get none-missing values
    hist_starrating = df.visitor_hist_starrating.isna()
    hist_adr = df.visitor_hist_adr_usd.isna()

    # Dit kan waarschijnlijk veel mooier en sneller maar het werkt...
    total_visited = []
    for index, row in df.iterrows():
        if hist_starrating[index] or hist_adr[index]:
            visited = 0
        else:
            visited = 1
        total_visited.append(visited)

    df["visited_before"] = total_visited
    
    return df

In [5]:
def price_quality(df):
    """
    Add a column of ratio price/quality to the DataFrame.
    """
    price_quality = []
    
    # Check for missing values
    for index, row in df.iterrows():
        price, quality =  row["price_usd"],  row["prop_starrating"]
        if price and quality:
            ratio = price / quality
        else:
            ratio = None
        price_quality.append(ratio)
        
    df["price_quality"] = price_quality
    
    # Replace missing values with median
    df["price_quality"].fillna((df["price_quality"].median()), inplace=True)
    
    return df 

In [6]:
def price_category(df):
    """
    Add a column of categories of price_usd and a column
    that corrected price for number of nights.
    Preprocessing of quantile cut showed that categories are:
    [(6.0889999999999995, 69.0] < (69.0, 90.0] < (90.0, 110.0] 
    < (110.0, 136.0] < (136.0, 170.077] < (170.077, 239.0] < (239.0, 554655.0]]
    
    """
    
    # Correct for number of nights ad add as new column
    df["price_correction"] = df["price_usd"] / df["srch_length_of_stay"]
    
    # Replace missing values
    df["price_correction"].fillna((df["price_correction"].median()), inplace=True)
    df["price_usd"].fillna((df["price_usd"].median()), inplace=True)
    
    # Make a new column of price categories
    df['PriceBand'] = pd.qcut(df["price_correction"], 7, labels=np.arange(1,8))
    
    return df

In [7]:
def process_remaining_cols(df):
    """
    Add some remaining (and interesting columns) to the dataframe.
    """
    
    # Replace missing values with median
    df["prop_brand_bool"].fillna((df["prop_brand_bool"].median()), inplace=True)
    df["random_bool"].fillna((df["random_bool"].median()), inplace=True)
    
    # Boolians
    df["prop_location_score1"].fillna((df["prop_location_score1"].mean()), inplace=True)
    df["prop_location_score2"].fillna((df["prop_location_score2"].mean()), inplace=True)
        
     # Replace missing values with mean
    df["promotion_flag"].fillna((df["promotion_flag"].mean()), inplace=True)
    
    return df

In [8]:
def add_score(df):
    """
    Add a score 
    """
    if df["booking_bool"] == 1 or df["click_bool"] == 1:
        score = 1
    else:
        score = 0
    return score


def preprocessing(traindf, dollarprice=True):
    traindf = competitors(traindf)
    traindf = visitor_history(traindf)
    traindf = price_quality(traindf)
    traindf = price_category(traindf)
    traindf = process_remaining_cols(traindf)
    
    # Add relevant columns
    df = traindf[["prop_id", "srch_id", "position", "price_quality", 
                  "competitor_bool", "availability_bools", "visited_before", 
                  "click_bool", "booking_bool", "PriceBand", 
                  "promotion_flag", "prop_brand_bool", "random_bool",
                  "prop_location_score1", "prop_location_score2"]]
    
    # Add relevant columns for trainingset
#     df = traindf[["prop_id", "srch_id", "price_quality", 
#                   "competitor_bool", "availability_bools", "visited_before", "PriceBand", 
#                   "promotion_flag", "prop_brand_bool", "random_bool",
#                   "prop_location_score1", "prop_location_score2"]]
    
    return df

In [10]:
# Oh jit kan niet met pandas Series werken, moet omgeschreven worden naar np arrays
# @jit(nopython=True)
df = preprocessing(traindf)
df.head(5)

,prop_id,srch_id,price_quality,competitor_bool,availability_bools,visited_before,PriceBand,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2
0,3180,1,39.666667,0,0,0,6,0,1,0,2.94,0.0691
1,5543,1,39.333333,0,0,0,6,0,1,0,2.64,0.0843
2,14142,1,24.500000,1,0,0,3,0,1,0,2.71,0.0556
3,22393,1,47.666667,0,0,0,6,0,1,0,2.40,0.0561
4,24194,1,26.333333,0,0,0,4,0,1,0,2.94,0.2090


In [11]:
df["price_quality"] = df.price_quality.astype(np.float32)
print(df.dtypes)
print()

if df.isnull().sum().sum() != 0:    
    print("\x1b[31mMissing values: \'\x1b[0m")
    print(df.isnull().sum())
else:
    print("\x1b[31mNo missing values!! :D \'\x1b[0m")

prop_id                    int64
srch_id                    int64
price_quality            float32
competitor_bool            int64
availability_bools         int64
visited_before             int64
PriceBand               category
promotion_flag             int64
prop_brand_bool            int64
random_bool                int64
prop_location_score1     float64
prop_location_score2     float64
dtype: object

No missing values!! :D '


## Totale dataset zonder missende waardes, hier kan RandomForestClassifier op gerund worden

In [12]:
filename = filename
if not os.path.exists(filename):
    df.to_pickle(filename)

# df.describe()